In [1]:
import tesserocr
from PIL import Image
import os
import numpy as np
import cv2
from tesserocr import PyTessBaseAPI, RIL
import shutil
import tempfile
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import subprocess
import shlex
import glob

In [2]:
# customised parameters
#image_path = '/data/tha+eng2.jpg'
#image_path = '/data/pk-1326.png'
lang_choice = 'tha+eng'
tessdata_prefix = "/data/tesseract-ralated/tesseract-ocr/tessdata"
output_dir = '/data/'
unichar_dir = output_dir
os.environ["TESSDATA_PREFIX"] = tessdata_prefix
# default params
RESIZE = 60

In [3]:
# extract lines
def getLineImages(image_path):
    image = Image.open(image_path)
    lines = []
    with PyTessBaseAPI(lang = lang_choice) as api:
        api.SetImage(image)
        boxes = api.GetComponentImages(RIL.TEXTLINE, True)
        #print(boxes)
        #print('Found {} box image components.'.format(len(boxes)))
        for i in range(len(boxes)):
            #print(i, 'th box')
            display(boxes[i][0])
            line = np.asarray(boxes[i][0])
            lines.append(line)
    return lines

# resize line to be the same height
def resizeLine(height, image):
    #image = np.dot(image[:], [0.299, 0.587, 0.114]) # change to grayscale
    ratio = height/image.shape[0]
    image = cv2.resize(image, (math.floor(ratio*image.shape[1]), height))
    image = image.astype(np.float32)
    return image

In [4]:
# get unichar and map accordingly
def getAlphabet(unichar_dir):
    num_cat = 2 + ord(u'\u007f')+1-ord(u'\u0021') + ord(u'\u0e7f')+1-ord(u'\u0e00')
    unichar_path = unichar_dir+'/charset_size={}.txt'.format(num_cat)
    index = 0
    if not os.path.exists(unichar_path):
        with open(unichar_path, 'w', encoding='utf-8') as f:
            f.write('0\t \n')
            for i in range(ord(u'\u0021'), ord(u'\u007f')+1):
                index += 1
                f.write(str(index)+'\t'+chr(i)+'\n')
            for i in range(ord(u'\u0e00'), ord(u'\u0e7f')+1):
                index += 1
                f.write(str(index)+'\t'+chr(i)+'\n')
            f.write('{}\t<nul>\n'.format(num_cat-1))
    alphabet = u''
    with open(unichar_path, 'r', encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines[:len(lines)-1]:
            alphabet += line.strip('\n').split('\t')[1]
    return alphabet, unichar_path

def text_to_label(text):
    if text=='<nul>':
        return len(alphabet)
    else:
        return alphabet.find(text)

def label_to_text(label):
    if label == len(alphabet):
        return '<nul>'
    else:
        return alphabet[label]

def cleanup(temp_dir):
    ''' Tries to remove the whole temp_dir '''
    try:
        shutil.rmtree(temp_dir)
    except OSError:
        pass

In [5]:
# create tfrecord for inference
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def createTFRecord(image_path, tempdir):
    # list of nparray of line images
    lines = getLineImages(image_path)
    # create a temp image.tfrecords
    basename = os.path.basename(image_path)
    tfrecords_filename = tempdir+'/'+str(len(lines))+'-'+os.path.splitext(basename)[0]+'.tfrecords'
    writer = tf.python_io.TFRecordWriter(tfrecords_filename)
    # write in tfrecord
    for i in range(len(lines)):
        text = ''
        label = [text_to_label(x) for x in text]
        blob = lines[i]
        if blob.shape[0]==0:
            continue
        img_resize = resizeLine(RESIZE, blob)
#         plt.imshow(img_resize, cmap='gray')
#         display(plt.show())
        _height, _width = img_resize.shape
        success, img_raw = cv2.imencode('.png', img_resize)
        img_raw = tf.compat.as_bytes(img_raw.tostring())
        example = tf.train.Example(features=tf.train.Features(
                feature={
                    'image/format': _bytes_feature(tf.compat.as_bytes('PNG')),
                    'image/encoded': _bytes_feature(img_raw),
                    #'image/class': _int64_list_feature(padLabels(label)),
                    'image/unpadded_class': _int64_list_feature(label),
                    'image/width': _int64_feature(_width),
                    'image/height': _int64_feature(_height),
                    'image/text': _bytes_feature(tf.compat.as_bytes(text))
                }))
        writer.write(example.SerializeToString())

    writer.close()
    return tfrecords_filename

In [13]:

# PYTHONIOENCODING=UTF-8 CUDA_VISIBLE_DEVICES="" python3 inference.py 
# --infer_data=/tmp/tmpg06wf32p/6-0-Tahoma.tfrecords  
# --decoder=/data/charset_size\=225.txt --train_dir=$train_dir

def inference(data_path, decoder_path, python_dir, train_dir, encoding='UTF-8', cuda_device=""):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device
    os.environ["PYTHONIOENCODING"] = encoding
    
    os.chdir(python_dir)
    command = []
    os.environ
    command += ('python3', 'inference.py',
                '--infer_data', data_path,
                '--decoder', decoder_path,
                '--train_dir', train_dir)
    result = subprocess.check_output(command)
    print(result.decode('utf-8'))
    
def my_predict(folder_dir):
    tempdir = tempfile.mkdtemp()
    infer_data = createTFRecord(image_path, tempdir)
    # run inference
    inference(infer_data, decoder_path, folder_dir+'python', folder_dir+'model')
    cleanup(tempdir)

In [16]:
# choose the image for inference
image_path = '/data/tha+eng3.png'
# customized params
folder_dir = '/data/street/'
alphabet, decoder_path = getAlphabet(unichar_dir)
# my model
import time
start = time.time()
my_predict(folder_dir)
print('spent: {:.2f}s'.format(time.time()-start))

มีภาพที่เริยกว่า ^The 16th moon" บนผนัง ฉันหวังว่ามันจะตำให้ห้องดูสวฤ

spent: 3.12s


In [15]:
# tesseract output
import pytesseract
start = time.time()
print(pytesseract.image_to_string(cv2.imread(image_path), lang=lang_choice))
print('spent: {:.2f}s'.format(time.time()-start))

ม ี ภา พ ท ี ่ เร ี ย ก ว ่ า “โท 6 16 ย ขา "ท ๐๐๓” บ น ผนัง ฉั น ห ว ั ง ว ่ า ม ั น จ ะ ท ํ า ให ้ ห ้ อ ง ด ู ส ว ย
spent: 0.73s
